In [1]:
import os
import json
import requests
from tqdm import tqdm
from bs4 import BeautifulSoup

### 1. Scraping the Menu

In [2]:
def scrape_menu(soup):
    
    menu = {}
    for sp in soup.find_all('div', class_ = '_2dS-v'):
        
        category_ = sp.find('h2').text.strip()
        temp = {}
        
        for prod in sp.find_all('div', class_ = '_2wg_t'):
            
            try:
                veg_or_non_veg_  = prod.find('p').text.split('.')[0].split(' ')[0]
            except:
                veg_or_non_veg_  = "NA"
                
            try:
                prod_name_       = prod.find('h3').text.strip()
            except:
                prod_name_       = "NA"
                
            try:
                prod_price_      = prod.find('span', class_ = 'rupee').text.strip()
            except:
                prod_price_      = "NA"
                
            temp[prod_name_] = {"price" : prod_price_, "veg_or_non_veg" : veg_or_non_veg_}
        menu[category_] = temp   
        
    return menu

### 2. Scraping Restaurent Details

In [3]:
def scrape_rest_details(link):
    
    soup = BeautifulSoup(requests.get(link).content,'html.parser')

    rest = {}
    
    try:
        rest_name    = soup.find('h1').text.strip()
    except:
        rest_name    = "NA"
    
    try:
        rating       = soup.find('div', class_ = '_1BpLF').find_all('div')[2].find_all('span')[0].text.strip()
    except:
        rating       = "NA"
    
    try:
        rating_count = soup.find('div', class_ = '_1BpLF').find_all('div')[2].find_all('span')[2].text.strip()
    except:
        rating_count = "NA"
    
    try:
        avg_cost     = soup.find('div', class_ = '_1BpLF').find_all('div')[2].find_all('span')[4].text.strip()
    except:
        avg_cost     = "NA"

    try:
        lic_no       = soup.find('li', class_ = '_167GT').text.split(' ')[-1]
    except:
        lic_no       = "NA"
        
    try:
        address      = soup.find_all('div', class_ = 'pr21h')[0].find_all('div')[-1].text
    except:
        address      = "NA"
        
    try:
        cuisine      = soup.find_all('div', class_ = 'pr21h')[1].find_all('div')[-1].text
    except:
        cuisine      = "NA"


    rest = {'name' : rest_name, 'rating' : rating, 'rating_count' : rating_count, 
            'cost' : avg_cost, 'address' : address, 'cuisine' : cuisine,
            'lic_no' : lic_no, 'menu' : scrape_menu(soup)}
    
    return rest

### 3. Scraping Data from All the Restaurents

In [4]:
fd = open('Checkpoints/data_548.json','r')
js = json.loads(fd.read())
fd.close()

In [5]:
check = 548

for city in tqdm(list(js.keys())[548:]):                               # Finding all the Cities
    if ('restaurants' in js[city].keys()):                             # Cities with no Subregions     
        if(js[city]['restaurants'].keys()):                            # If no restauraent in a city
            for r in tqdm(js[city]['restaurants'].keys()):             # Going through each restaurent           
                try:
                    rest_link_     = "https://www.swiggy.com/restaurants/" + js[city]['restaurants'][r]['link']
                    temp_dct         = scrape_rest_details(rest_link_) # Scraping Restaurent Data
                    temp_dct["link"] = rest_link_                      # Adding Restaurent Link
                    js[city]['restaurants'][r] = temp_dct              # Adding details in the Database
                except:
                    pass
    else:                                                              # Cities with Subregions     
        for sub in tqdm(js[city].keys()):                              # Going through all the keys
            if ('link' != sub):                                        # Taking keys except link (SubRegions)
                for r in js[city][sub]['restaurants'].keys():          # Passin each restaurent in that subregion
                    try:
                        rest_link_ = 'https://www.swiggy.com/restaurants/' + js[city][sub]['restaurants'][r]['link']
                        temp_dct   = scrape_rest_details(rest_link_)      # Scraping Restaurent Data
                        temp_dct["link"] = rest_link_                     # Adding Restaurent Link
                        js[city][sub]['restaurants'][r] = temp_dct        # Saving the Data in the database
                    except:
                        pass

    check += 1                                                        # Saving Checkpoins
    
    fd = open('Checkpoints/data_'+str(check)+'.json','w')
    fd.write(json.dumps(js))
    fd.close()

 45%|███████████████████▌                       | 35/77 [01:03<00:43,  1.03s/it]


 76%|████████████████████████████████▌          | 53/70 [01:34<00:12,  1.37it/s]


 35%|███████████████                            | 32/91 [00:54<00:49,  1.19it/s]


 20%|████████▎                                 | 36/182 [01:17<01:33,  1.56it/s]


 73%|█████████████████████████████▉           | 133/182 [03:40<00:32,  1.49it/s]


...

 96%|███████████████████████████████████████▎ | 160/167 [06:39<00:06,  1.08it/s]


 91%|███████████████████████████████████████    | 88/97 [03:22<00:47,  5.23s/it]


 33%|█████████████▉                            | 59/178 [01:47<01:15,  1.58it/s]


 88%|███████████████████████████████████▉     | 156/178 [04:52<01:54,  5.21s/it]


100%|████████████████████████████████████████| 75/75 [7:43:58<00:00, 371.18s/it]


In [ ]:
rest_link_

In [ ]:
js['Bangalore'].keys()

In [7]:
import os
os.listdir('Checkpoints/')

['data_336.json']